In [102]:
import numpy as np
import pandas as pd

In [103]:
Constructeurs = pd.read_csv(r"Donnees propres\Constructeurs.csv", sep=',').drop(columns=['Unnamed: 0'])
Constructeurs.columns

Index(['raceId', 'constructorId', 'winsConstructeur'], dtype='object')

In [104]:
Driver = pd.read_csv(r"Donnees propres/DriverALL.csv",sep=',').drop(columns=['Unnamed: 0'])
Driver.columns

Index(['raceId', 'driverId', 'winsDriver', 'dob'], dtype='object')

In [105]:
Meteo = pd.read_csv(r'Donnees propres\Meteo.csv', sep=',').drop(columns=['Unnamed: 0'])
Meteo.columns

Index(['raceId', 'year', 'GradTemperature', 'GradSec', 'weather_cloudy'], dtype='object')

In [106]:
Races = pd.read_csv(r'Donnees propres\Races.csv', sep=',').drop(columns=['Unnamed: 0'])
Races.columns

Index(['raceId', 'year', 'round', 'circuitId', 'monthRace', 'startRace'], dtype='object')

In [107]:
Resultat = pd.read_csv(r'Donnees propres\resultatsCourse.csv',sep=',').drop(columns=['Unnamed: 0'])
Resultat.columns

Index(['raceId', 'driverId', 'constructorId', 'grid', 'winner', 'IsSecond',
       'IsThird'],
      dtype='object')

In [108]:
Qualif = pd.read_csv(r'Donnees propres\Qualif.csv', sep=',').drop(columns=['Unnamed: 0'])
Qualif.columns 

Index(['raceId', 'driverId', 'constructorId', 'q1', 'TempsMaxQualif',
       'TempsMax-q1'],
      dtype='object')

In [109]:
#le premier modèle porte sur les caractéristiques suivantes : wins driver, grid, q1, year, ageAtrace
Column_model1 = ['raceId', 'driverId', 'grid','AgeAtRace', 'year', 'q1', 'winner']

DataM1 = pd.merge(Resultat[['raceId', 'driverId','grid', 'winner']],Driver[['raceId', 'driverId', 'winsDriver', 'dob']], on=['raceId', 'driverId'])
DataM1

,raceId,driverId,grid,winner,winsDriver,dob
0,18,1,1,1,0,1985
1,18,2,5,0,0,1977
2,18,3,7,0,0,1985
3,18,4,11,0,0,1981
4,18,5,3,0,0,1981
...,...,...,...,...,...,...
25366,1096,854,12,0,0,1999
25367,1096,825,16,0,0,1992
25368,1096,1,5,0,0,1985
25369,1096,849,20,0,0,1995


In [110]:
DataM1 = pd.merge(DataM1, Races[['raceId','year']], on=['raceId'])
DataM1['AgeAtRace'] = DataM1['year'] - DataM1['dob']
DataM1.columns

Index(['raceId', 'driverId', 'grid', 'winner', 'winsDriver', 'dob', 'year',
       'AgeAtRace'],
      dtype='object')

In [111]:
def Data_X_et_Y(Set_Data, ColumnsToKeep_X,ColumnsToKeep_Y):
    return Set_Data[ColumnsToKeep_X].set_index(['raceId', 'driverId']), Set_Data[ColumnsToKeep_Y].set_index(['raceId', 'driverId'])

In [112]:
#modèle 1 : simple une regression logistique qui alimentera un random forest
'raceId', 'driverId', 'grid', 'winner', 'winsDriver', 'dob', 'year','AgeAtRace'

('raceId',
 'driverId',
 'grid',
 'winner',
 'winsDriver',
 'dob',
 'year',
 'AgeAtRace')

In [139]:
DataM1_train = DataM1[DataM1['year']<=2020]
DataM1_test = DataM1[DataM1['year']==2021]
Columns_M1_X = ['raceId', 'driverId','grid', 'winsDriver', 'dob', 'year','AgeAtRace']
Columns_M1_Y = ['raceId', 'driverId','winner']
DataM1_train_X, DataM1_train_Y = Data_X_et_Y(DataM1_train,Columns_M1_X,Columns_M1_Y)
DataM1_test_X, DataM1_test_Y = Data_X_et_Y(DataM1_test,Columns_M1_X,Columns_M1_Y)

In [140]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [143]:
#on fait le gradient boost
M1_Logistic = LogisticRegression()
M1_Logistic_Gris = {
    'penalty' : ['l2'],
    'solver' : ['liblinear'],
    'tol' : [0.001],
    'max_iter' : [8000],
    'random_state' : [14]
}

M1_Logistic_Gris = GridSearchCV(M1_Logistic, M1_Logistic_Gris,scoring='recall')

M1_Logistic_Gris.fit(DataM1_train_X, DataM1_train_Y.values.ravel())
M1_Logistic = LogisticRegression(**M1_Logistic_Gris.best_params_)
M1_Logistic = M1_Logistic.fit(DataM1_train_X, DataM1_train_Y.values.ravel())
M1_Logistic.score(DataM1_train_X, DataM1_train_Y.values.ravel())

0.9583928790167817

In [116]:
def FonctionProba_to_Score(Modele, Data_set_X, Data_set_Y):
    Y_predit = Data_set_Y
    probabilite = Modele.predict_proba(Data_set_X)
    probabilite = pd.DataFrame(probabilite).drop(columns=(0))
    probabilite = np.array(probabilite)
    Y_predit['PREpositionOrder'] = probabilite

    Top = []
    Podium = []
    
    Courses = Y_predit.index.get_level_values("raceId").unique()
    for Race in Courses:
        
        Drivers = Y_predit['PREpositionOrder'][Race].index.get_level_values("driverId")
        NB_driver = len(Drivers)
        Classement = np.array(Y_predit["PREpositionOrder"][Race])
        Classement = (NB_driver) - (Classement.argsort().argsort())
        count = 0
        for Conducteur in Drivers:
            if Classement[count] <= 3:
                Podium.append(Classement[count])
                if Classement[count] == 1:
                    Top.append(1)
                else:
                    Top.append(0)
            else:
                Podium.append(0)
                Top.append(0)
            count += 1
    Y_predit.loc[:,'winner'] = Top
    Y_predit.loc[:,'positionOrder'] = Podium
    return Y_predit

In [144]:
DataM1_test_Y = FonctionProba_to_Score(M1_Logistic, DataM1_test_X,DataM1_test_Y)

In [145]:
DataM1_test_Y = DataM1_test_Y.drop(columns=['PREpositionOrder'])
DataM1_test_Y.to_csv('Model1_propre/ResultatLogistic_V1.csv') #69.88

on va utiliser les resultats du modèle logistique pour faire un modele random forest 
on va faire une matrice train avec les données de base et les résultats de logistique
DataM1_train_X
DataM1_train_Y
DataM1_test_X
DataM1_test_Y

In [146]:
def FromIndexToColumn(DataWithIndex):
    matrice = DataWithIndex
    for i in DataWithIndex.index.names:
        matrice[i] = matrice.index.get_level_values(i)
    matrice = matrice.reset_index(level=DataWithIndex.index.names, drop=True)
    return matrice

def fusionnerDeuxMatrice(DataArray, DataResult):
    Matrice = pd.merge(DataArray, DataResult, on=['raceId', 'driverId'])
    return Matrice

DataM1_train

In [147]:
#on va fusionner les matrices résultats et avec les caractéristiques
DataM1_Train_FromLogistic = FromIndexToColumn(FonctionProba_to_Score(M1_Logistic, DataM1_train_X,DataM1_train_Y).rename(columns={'winner':'expectedWinnerLogistic', 'PREpositionOrder' : 'expectedPREpositionOrderLogistic', 'positionOrder' : 'expectedpositionOrderLogistic'}))

DataM1_Train_FromLogistic = fusionnerDeuxMatrice(DataM1_train,DataM1_Train_FromLogistic)
Columns_M1_X_RF = ['raceId', 'driverId','grid', 'winsDriver', 'dob', 'year','AgeAtRace','expectedWinnerLogistic','expectedPREpositionOrderLogistic','expectedpositionOrderLogistic']
Columns_M1_Y_RF = ['raceId', 'driverId','winner']
DataM1_Train_FromLogistic_X, DataM1_Train_FromLogistic_Y = Data_X_et_Y(DataM1_Train_FromLogistic,Columns_M1_X_RF,Columns_M1_Y_RF)

In [148]:
M1_RandomForest = RandomForestClassifier()
M1_RandomForest_Grid = {
    'n_estimators' : [40],
    'criterion' : ["gini"],
    'max_depth' : [10],
    'random_state' : [14]
}
M1_RandomForest_Grid = GridSearchCV(M1_RandomForest,M1_RandomForest_Grid,scoring='recall')
M1_RandomForest_Grid = M1_RandomForest_Grid.fit(DataM1_Train_FromLogistic_X,DataM1_Train_FromLogistic_Y.values.ravel())
M1_RandomForest = RandomForestClassifier(**M1_RandomForest_Grid.best_params_)
M1_RandomForest.fit(DataM1_Train_FromLogistic_X,DataM1_Train_FromLogistic_Y.values.ravel())
M1_RandomForest.score(DataM1_Train_FromLogistic_X,DataM1_Train_FromLogistic_Y.values.ravel())

0.9711565728175005

In [149]:
#maintenant qu'on le modèle fir on run les data du test  : DataM1_test
DataM1_Test_FromLogistic = FromIndexToColumn(FonctionProba_to_Score(M1_Logistic, DataM1_test_X,DataM1_test_Y).rename(columns={'winner':'expectedWinnerLogistic', 'PREpositionOrder' : 'expectedPREpositionOrderLogistic', 'positionOrder' : 'expectedpositionOrderLogistic'}))

DataM1_Test_FromLogistic = fusionnerDeuxMatrice(DataM1_test,DataM1_Test_FromLogistic)
Columns_M1_X_RF = ['raceId', 'driverId','grid', 'winsDriver', 'dob', 'year','AgeAtRace','expectedWinnerLogistic','expectedPREpositionOrderLogistic','expectedpositionOrderLogistic']
Columns_M1_Y_RF = ['raceId', 'driverId','winner']
DataM1_Test_FromLogistic_X, DataM1_Test_FromLogistic_Y = Data_X_et_Y(DataM1_Test_FromLogistic,Columns_M1_X_RF,Columns_M1_Y_RF)


In [150]:
DataM1_test_Y_RF = FonctionProba_to_Score(M1_RandomForest,DataM1_Test_FromLogistic_X, DataM1_Test_FromLogistic_Y)
DataM1_test_Y_RF = DataM1_test_Y_RF.drop(columns="PREpositionOrder")

In [151]:
DataM1_test_Y_RF.to_csv('Model1_propre/ResultatRF_V1.csv') #68.06

On a des résultats qui stagnent en dessous des 70%
Passons maintenant à la prise en compte des qualifications on va tenter une regression logistique
Races, Qualif, Resultat
races : 'raceId', 'year', 'round', 'circuitId', 'monthRace', 'startRace'
Qualif : 'raceId', 'driverId', 'constructorId', 'q1', 'TempsMaxQualif', 'TempsMax-q1'
Resultat : 'raceId', 'driverId', 'constructorId', 'grid', 'winner', 'IsSecond', 'IsThird'

In [168]:
Resultat

,raceId,driverId,constructorId,grid,winner,IsSecond,IsThird
0,18,1,1,1,1,0,0
1,18,2,2,5,0,1,0
2,18,3,3,7,0,0,1
3,18,4,4,11,0,0,0
4,18,5,1,3,0,0,0
...,...,...,...,...,...,...,...
25835,1096,854,210,12,0,0,0
25836,1096,825,210,16,0,0,0
25837,1096,1,131,5,0,0,0
25838,1096,849,3,20,0,0,0


In [213]:
Data_Logistic_M2 = pd.merge(Resultat[['raceId', 'driverId', 'constructorId', 'grid', 'winner']],Races, on =['raceId'], how='outer')
Data_Logistic_M2 = pd.merge(Data_Logistic_M2, Qualif[['raceId', 'driverId', 'constructorId', 'q1', 'TempsMaxQualif', 'TempsMax-q1']], on=['raceId','driverId'], how='outer')
Data_Logistic_M2.columns

Index(['raceId', 'driverId', 'constructorId_x', 'grid', 'winner', 'year',
       'round', 'circuitId', 'monthRace', 'startRace', 'constructorId_y', 'q1',
       'TempsMaxQualif', 'TempsMax-q1'],
      dtype='object')

In [223]:
Columns_M2_X = ['raceId', 'driverId', 'grid', 'year', 'round', 'circuitId', 'monthRace', 'startRace', 'q1', 'TempsMaxQualif', 'TempsMax-q1']
Columns_M2_Y = ['raceId', 'driverId','winner']
Data_Logistic_M2_train = Data_Logistic_M2[Data_Logistic_M2['year']<= 2020].dropna(axis=0)
Data_Logistic_M2_test = Data_Logistic_M2[Data_Logistic_M2['year'] == 2021].fillna(0)

DataM2_train_X, DataM2_train_Y = Data_X_et_Y(Data_Logistic_M2_train,Columns_M2_X,Columns_M2_Y)
DataM2_test_X, DataM2_test_Y = Data_X_et_Y(Data_Logistic_M2_test,Columns_M2_X,Columns_M2_Y)

In [231]:
import xgboost

In [243]:
Model2_Logistic = LogisticRegression()
Model2_Logistic_Grid = {
    'penalty' : ['l2'],
    'solver' :  ['lbfgs'],
    'max_iter' : [4000],
    'random_state' : [14]
}

Model2_Logistic_Grid = GridSearchCV(Model2_Logistic, Model2_Logistic_Grid, scoring='recall')
Model2_Logistic_Grid.fit(DataM2_train_X, DataM2_train_Y.values.ravel())
Model2_Logistic = LogisticRegression(**Model2_Logistic_Grid.best_params_)
Model2_Logistic.fit(DataM2_train_X, DataM2_train_Y.values.ravel())
Model2_Logistic.score(DataM2_train_X, DataM2_train_Y.values.ravel())

0.9510119595216191

In [244]:
DataM2_test_Y = FonctionProba_to_Score(Model2_Logistic,DataM2_test_X,DataM2_test_Y).drop(columns=['PREpositionOrder'])
DataM2_test_Y
DataM2_test_Y.to_csv('Model2_propre/LogisticDriverV1.csv')

In [245]:
DataM2_test_Y

winner  positionOrder
raceId driverId                       
1051   1              1              1
       4              0              3
       8              0              0
       20             0              0
       815            0              0
...                 ...            ...
1073   847            0              0
       849            0              0
       852            0              0
       853            0              0
       854            0              0

[440 rows x 2 columns]